Initialisation

In [ ]:
import numpy as np
import numpy.random as rd
import matplotlib.pyplot as plt
import time

N = 10 #Number of points

Dataset generation

In [ ]:
def pointgeneration():
    """
    Return a list of two numbers in [0,1]
    """
    return rd.rand(2)

In [ ]:
def generateB(n):
    """
    Return list of n list of 2 elements
    """
    return rd.uniform(size=(n, 2))

In [ ]:
def generateA(n):

    A=generateB(n-4)

    #adding the corner
    A= np.append(A,[[0,0]],axis=0)
    A= np.append(A,[[0,1]],axis=0)
    A= np.append(A,[[1,0]],axis=0)
    A= np.append(A,[[1,1]],axis=0)

    #rotating the set
    angle = rd.uniform() * 2 * np.pi
    rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
    A = A @ rotation_matrix

    # Shuffle the points
    np.random.shuffle(A)
    return A


In [ ]:
def generateC(n,r):
    C=[]
    for i in range(n):
        ri=rd.uniform()*r
        ai = rd.uniform()*2*np.pi
        C.append( np.array([ri*np.cos(ai),ri*np.sin(ai)]))
    return np.array(C)

In [ ]:
def generateD(n,r):
    D=[]
    for i in range(n):
        angle = rd.uniform()*2*np.pi
        D.append( np.array([r*np.cos(angle),r*np.sin(angle)]))
    return np.array(D)


Sweeping

In [ ]:
def clockwise(a,b,c):
    #Takes 3 points a,b,c of type np.array and return if the triangle ABC is clock-wised 
    
    if np.linalg.det(np.concatenate(((c-b).reshape(2,1),(b-a).reshape(2,1)),axis=1))>0: return True
    else: return False

In [ ]:
def sweeping_algo(l):
    #l is a list of points of type np.array, assuming there are >= 2 points and sorted by increasing x
    order=l[:, 0].argsort()
    l=l[order]
    N=l.shape[0]

    hull=[]

    hull.append(l[0])
    hull.append(l[1])

    for i in (range(2,N)):
        if clockwise(hull[-2],hull[-1],l[i]): hull.append(l[i])
        else : 
            while len(hull)>=2 and not clockwise(hull[-2],hull[-1],l[i]): 
                hull.pop()
            hull.append(l[i])

    hull2=[]
    hull2.append(l[N-1])
    hull2.append(l[N-2])

    for i in reversed(range(N-2)):
        if clockwise(hull2[-2],hull2[-1],l[i]): hull2.append(l[i])
        else : 
            while len(hull2)>=2 and not clockwise(hull2[-2],hull2[-1],l[i]): 
                hull2.pop()
            hull2.append(l[i])
    
    return np.array(hull+hull2)


Test Sweeping

In [ ]:
def sweeping_test(N):
    c=generateC(N,1)

    cHull=sweeping_algo(c)

    plt.plot(c[:,0],c[:,1],'xb')
    plt.plot(cHull[:,0],cHull[:,1],'-r')

    plt.show()


In [ ]:
def complexity_sweeping(N):
    R=range(int(1e6),N,5*int(1e6))
    T=[]
    for i in R:
        start_time = time.time()
        sweeping_algo(generateC(i,1))
        T.append( time.time() - start_time )
    plt.plot(R,T,'-r')
    plt.show()

In [ ]:
complexity_sweeping(int(1e8))

Output-sensitive algorithm 

Median 

In [ ]:
def median(L):
    if len(L)<=5:
        return median_sous_liste(L)
    else:
        return selection(L, len(L)//2)

def selection(L,k):
    if len(L)<=5:
        return median_sous_liste(L)
    else:
        liste_of_mediane=[median_sous_liste(L[i:i+5]) for i in range(0, len(L), 5)]
        mediane_of_mediane = selection(liste_of_mediane, len(liste_of_mediane)//2)
        
        under = [x for x in L if x<mediane_of_mediane]
        over = [x for x in L if x>= mediane_of_mediane]
        
        if k < len(under):
            return selection(under, k)
        elif k == len(under):
            return mediane_of_mediane
        else:
            return selection(over, k - len(under) - 1)


def median_sous_liste(L):
    L = np.sort(L)
    if len(L)==5 :
        return L[2]
    elif len(L) == 4 :
        return (L[1]+L[2])/2
    elif len(L)== 3 :
        return L[1]
    elif len(L) == 2 : 
        return (L[0]+L[1])/2
    else :
        return L[0]